In [2]:
from pyomo.environ import *
from pyomo.dae import *

m = ConcreteModel()

m.t = ContinuousSet(bounds=(0,1)) 

m.x1 = Var(m.t, bounds=(-10,10))
m.x2 = Var(m.t, bounds=(0,10))
m.u = Var(m.t, initialize=0, bounds=(-10,10))

T1 = 0.1
T2 = 0.5
m.x1dot = DerivativeVar(m.x1)
m.x2dot = DerivativeVar(m.x2)

m.obj = Objective(expr=(m.x2[1]-2)**2)

def _obj(M):
    #e=0.0
    #x=0
    #for i in range(len(M.u)):

    #    e += M.u[i]**2
    #for i in range(len(M.x2)):
    #    x += (M.x2[i]-3)**2
    #e = Integral(expr=M.u, wrt=M.u, bounds=(0,1))
    #x = Integral(expr=(m.x2-2)**2, wrt=M.x1, bounds=(0,1))
    return sum(((M.x1[i]-5)**2+
               (M.x2[i]-5)**2+
               (M.u[i])**2) for i in M.t) 
m.obj = Objective(rule=_obj, sense=minimize, doc='Define objective function')

def _x1dot(M,i):
    if i == 0:
        return Constraint.Skip
    return M.x1dot[i] ==1/T1*(M.u[i] - M.x1[i] )

m.x1dotcon = Constraint(m.t, rule=_x1dot)

def _x2dot(M,i):
    if i == 0:
        return Constraint.Skip
    return M.x2dot[i] == 1/T2*(M.x1[i] -  M.x2[i])
m.x2dotcon = Constraint(m.t, rule=_x2dot)

def _init(M):
    yield M.x1[0] == 0
    yield M.x2[0] == 0
    yield ConstraintList.End
m.init_conditions = ConstraintList(rule=_init)

from pyomo.environ import *
from pyomo.dae import *
from pyomo.opt import SolverFactory
from pyomo.dae.plugins.finitedifference import Finite_Difference_Transformation
from pyomo.dae.plugins.colloc import Collocation_Discretization_Transformation


instance = m

# Discretize model using Backward Finite Difference method
#discretize = Finite_Difference_Transformation()
#disc_instance = discretize.apply(instance,nfe=20,scheme='BACKWARD')

# Discretize model using Orthogonal Collocation
discretize = Collocation_Discretization_Transformation()
disc_instance = discretize.apply(instance,nfe=20,ncp=10,scheme='LAGRANGE-RADAU')
# Will reimplement this method in future release of pyomo.dae
# disc_instance = discretize.reduce_collocation_points(var=instance.u,
# 	ncp=2, diffset=instance.t)

solver='ipopt'
opt=SolverFactory(solver)

results = opt.solve(disc_instance,tee=True)
disc_instance.load(results)

x1 = []
x2 = []
u = []
t=[]

print(sorted(disc_instance.t))

for i in sorted(disc_instance.t):
    t.append(i)
    x1.append(value(disc_instance.x1[i]))
    x2.append(value(disc_instance.x2[i]))
    u.append(value(disc_instance.u[i]))


    


	This is usually indicative of a modelling error.
	To avoid this warning, use block.del_component() and block.add_component().
	Please use either Transformation.apply_to() for in-place transformations
	or Transformation.create_using() for transformations that create a new,
	independent transformed model instance.


AttributeError: 'NoneType' object has no attribute '_transformation_data'

In [73]:
import matplotlib.pyplot as plt

plt.plot(t,x1)

plt.plot(t,x2) 
plt.show()

plt.plot(t,u) 
plt.show()


43